In [1]:
from dataset import create_dataset, create_sampler, create_loader
import argparse
import ruamel.yaml as yaml
import torch
import numpy as np
from matplotlib import pyplot as plt
import json

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def img_show(img):
    img = torch.permute(img,dims=(1,2,0)).detach().numpy()
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    plt.imshow(img)
    plt.axis(False)
    plt.show()

In [3]:
parser = argparse.ArgumentParser()     
parser.add_argument('--config', default='./configs/Retrieval_coco_romix.yaml')
parser.add_argument('--output_dir', default='output/Retrieval_full_romix_wj')        
parser.add_argument('--checkpoint', default='')   
parser.add_argument('--text_encoder', default='bert-base-uncased')
parser.add_argument('--evaluate', action='store_true')
parser.add_argument('--device', default='cuda')
parser.add_argument('--seed', default=42, type=int)
parser.add_argument('--world_size', default=1, type=int, help='number of distributed processes')    
parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
parser.add_argument('--distributed', default=True, type=bool)
args = parser.parse_args([])

config = yaml.load(open(args.config, 'r'), Loader=yaml.Loader)
config['output_dir'] = args.output_dir

In [4]:
train_dataset, val_dataset, test_dataset = create_dataset('re', config)  

/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:891: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [ ]:
img_info_dict = json.load(open('data/COCO/Annotations/coco_img_info.json', 'r'))
train_file_dict = json.load(open('data/COCO/Annotations/coco_train.json', 'r'))

In [ ]:
train_file_keys = list(str(item['image_id'].split('_')[-1]) for item in train_file_dict)

In [17]:
for i, (image, text, idx) in enumerate(train_dataset):
    # when loading is fail print the idx
    if i % 1000 == 0:
        print(i)

0


KeyboardInterrupt: 

In [20]:
train_dataset[1850].text

AttributeError: 'tuple' object has no attribute 'text'

In [ ]:
img_info_dict['318219']['captions']

['A young boy standing in front of a computer keyboard.',
 'a little boy wearing headphones and looking at a computer monitor',
 'He is listening intently to the computer at school.',
 'A young boy stares up at the computer monitor.',
 'a young kid with head phones on using a computer ']

In [ ]:
temp_list =[]
for keys in train_file_keys:
    if keys not in train_dataset.romixgen.image_info_dict.keys():
        temp_list.append(keys)

In [ ]:
len(train_dataset.romixgen.image_info_dict.keys())

45886

In [ ]:
train_dataset.romixgen.obj_bg_dict.keys()

dict_keys(['obj', 'bg', 'unusable_bg'])

In [ ]:
len(train_dataset.romixgen.obj_bg_dict['unusable_bg'])

974

In [ ]:
len(train_dataset.romixgen.obj_bg_dict['bg'])

2063

In [ ]:
len(train_dataset.romixgen.obj_bg_dict['obj'])

42849

In [ ]:
len(train_file_keys)

566747

In [ ]:
'318219' in train_file_keys

True

In [ ]:
train_dataset.romixgen.raw_image_info

'data/COCO/Annotations/coco_img_info_small.json'

In [ ]:
img_info_dict['318219']

{'file_name': 'COCO_val2014_000000318219.jpg',
 'width': 556,
 'height': 640,
 'objects_count': 11,
 'max_obj_id': 491276,
 'max_obj_cat_id': 1,
 'max_obj_super_cat': ['person', 'people'],
 'max_obj_cat': ['person', 'people'],
 'max_obj_area': 126863.98395,
 'max_obj_midpoint': [249.84, 339.40000000000003],
 'max_obj_bbox': [40.65, 38.8, 418.38, 601.2],
 'max_obj_segment_points': [[41.52,
   617.23,
   47.25,
   598.62,
   57.27,
   581.44,
   42.95,
   567.12,
   45.82,
   549.94,
   57.27,
   531.32,
   60.13,
   519.87,
   74.45,
   502.69,
   77.32,
   496.96,
   57.27,
   474.05,
   48.68,
   436.83,
   57.27,
   419.65,
   70.16,
   319.42,
   88.77,
   270.74,
   104.52,
   250.7,
   107.38,
   249.27,
   105.95,
   220.63,
   114.54,
   217.77,
   118.84,
   214.9,
   84.47,
   179.11,
   65.86,
   146.18,
   61.57,
   103.23,
   68.72,
   78.89,
   81.61,
   61.7,
   98.79,
   51.68,
   134.59,
   38.8,
   169.61,
   40.79,
   199.02,
   53.11,
   210.47,
   57.41,
   220.49,


In [ ]:
temp_ann = {'image_id':"coco_"+"318219"}

In [ ]:
train_dataset.romixgen(temp_ann)

KeyError: '318219'

In [ ]:
train_dataset.romixgen.image_info_dict

{'3580': {'file_name': 'COCO_val2014_000000003580.jpg',
  'width': 640,
  'height': 480,
  'max_obj_super_cat': ['person', 'people'],
  'max_obj_cat': ['person', 'people'],
  'max_obj_area': 137299.84804999997,
  'max_obj_midpoint': [320.0, 279.295],
  'max_obj_bbox': [0.0, 83.86, 640.0, 390.87],
  'max_obj_area_portion': 0.4469396095377603,
  'captions': ['A man sitting on a touch holding a game controller.',
   'A man sitting on a sofa holding a game controller.',
   'Person with a device sitting in a living room.',
   'A man sitting on a couch playing with a game system.',
   'A man lounging on a sofa holding a Wii remote.'],
  'obj_bg': 'obj'},
 '100974': {'file_name': 'COCO_val2014_000000100974.jpg',
  'width': 640,
  'height': 322,
  'max_obj_super_cat': ['vehicle', 'vehicles'],
  'max_obj_cat': ['airplane', 'airplanes'],
  'max_obj_area': 31451.592950000006,
  'max_obj_midpoint': [294.68, 208.14999999999998],
  'max_obj_bbox': [2.77, 127.1, 583.82, 162.1],
  'max_obj_area_portio